In [1]:
import pandas as pd
import math
import match_helper as helper

In [6]:
from imp import reload
reload(helper)
dbaccess = helper.get_dbaccess()
buyer_access = dbaccess + ['buyer']
manufacturer_access = dbaccess + ['manufacturer']
shared_access = dbaccess + ['shared']
crm_access = dbaccess + ['crm']

/Users/glen/Desktop/SQL external python/


In [7]:
### Buyer

In [8]:
reload(helper)
buyer_sql = """
SELECT
          c.id_company
        , c.name AS company_name
        , country.name AS company_country
        , c.city AS company_city
        , CASE WHEN c.fk_company_status = 1 THEN 'Active' ELSE 'Inactive' END AS company_status
        , c.created_at AS company_created_at
        , to_char(c.created_at::TIMESTAMP WITH TIME ZONE, 'yyyy"M"mm') AS cohort
        , cb.id_company_brand
        , cb.name AS brand_name
        , CASE WHEN cb.fk_company_brand_status = 1 THEN 'Active' ELSE 'Inactive' END AS brand_status
        , cbpo.samples
        , cbpo.out_of_stock
        , cbpo.special_order
        , cbpo.standard_order
        , cbpos.min_qty
        , cbpos.avg_qty
        , cbpos.max_qty
FROM company AS c
LEFT JOIN country ON country.id_country = c.fk_country
LEFT JOIN company_brand AS cb ON cb.fk_company = c.id_company
LEFT JOIN company_brand_po AS cbpo ON cbpo.fk_company_brand = cb.id_company_brand
LEFT JOIN company_brand_po_size AS cbpos ON cbpos.fk_company_brand = cb.id_company_brand
ORDER BY c.id_company, cb.id_company_brand;
"""
b_df = helper.query_retrieval(buyer_sql, buyer_access)

In [9]:
b_df[:5]


,id_company,company_name,company_country,company_city,company_status,company_created_at,cohort,id_company_brand,brand_name,brand_status,samples,out_of_stock,special_order,standard_order,min_qty,avg_qty,max_qty
0,1,Foursource Buying SE,Germany,Berlin,Inactive,2017-02-01 16:05:47.881287+00:00,2017M02,1.0,Mama's Beste,Inactive,0.0,4.0,4000.0,4000000.0,300.0,20000.0,30000000.0
1,1,Foursource Buying SE,Germany,Berlin,Inactive,2017-02-01 16:05:47.881287+00:00,2017M02,7.0,Lisa Leuchten,Inactive,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,Foursource Buying SE,Germany,Berlin,Inactive,2017-02-01 16:05:47.881287+00:00,2017M02,8.0,The Wall Brand,Inactive,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,Foursource Buying SE,Germany,Berlin,Inactive,2017-02-01 16:05:47.881287+00:00,2017M02,9.0,Lissy Lamp,Inactive,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,Foursource Buying SE,Germany,Berlin,Inactive,2017-02-01 16:05:47.881287+00:00,2017M02,13.0,The Wall,Inactive,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
buyer_ct_sql = """
SELECT
          cbo.fk_company_brand
        , ct.id_category_tree AS product_range_code
        , ct1.name || ' - ' || ct2.name || ' - ' || ct3.name || ' - ' || pg1.name || ' - ' || pg2.name   AS product_range 
        , pg2.active
        , pg2.verified
FROM company_brand_offering AS cbo 
LEFT JOIN category_tree AS ct ON ct.id_category_tree = cbo.fk_category_tree
LEFT JOIN ct1 ON ct.fk_ct1 = ct1.id_ct1
LEFT JOIN ct2 ON ct.fk_ct2 = ct2.id_ct2
LEFT JOIN ct3 ON ct.fk_ct3 = ct3.id_ct3
LEFT JOIN pg1 ON ct.fk_pg1 = pg1.id_pg1
LEFT JOIN pg2 ON ct.fk_pg2 = pg2.id_pg2
ORDER BY cbo.fk_company_brand;
"""
b_ct_df = helper.query_retrieval(buyer_ct_sql, shared_access)

In [11]:
b_ct_df[:5]


,fk_company_brand,product_range_code,product_range,active,verified
0,1,1967,Apparel - Active - Women - Trousers - Hiking P...,True,True
1,1,1946,Apparel - Active - Women - Accessories - Cap,True,True
2,1,334,Apparel - Active - Women - Jackets & Coats - P...,True,True
3,1,2045,Apparel - Casual - Kids - Trousers - Pants,True,True
4,1,361,Apparel - Active - Women - Trousers - Bib Shorts,True,True


In [12]:
b_joined = pd.merge(b_df, b_ct_df, left_on='id_company_brand', right_on='fk_company_brand')


In [13]:
b_joined[:10]


,id_company,company_name,company_country,company_city,company_status,company_created_at,cohort,id_company_brand,brand_name,brand_status,...,special_order,standard_order,min_qty,avg_qty,max_qty,fk_company_brand,product_range_code,product_range,active,verified
0,1,Foursource Buying SE,Germany,Berlin,Inactive,2017-02-01 16:05:47.881287+00:00,2017M02,1,Mama's Beste,Inactive,...,4000.0,4000000.0,300.0,20000.0,30000000.0,1,1967,Apparel - Active - Women - Trousers - Hiking P...,True,True
1,1,Foursource Buying SE,Germany,Berlin,Inactive,2017-02-01 16:05:47.881287+00:00,2017M02,1,Mama's Beste,Inactive,...,4000.0,4000000.0,300.0,20000.0,30000000.0,1,1946,Apparel - Active - Women - Accessories - Cap,True,True
2,1,Foursource Buying SE,Germany,Berlin,Inactive,2017-02-01 16:05:47.881287+00:00,2017M02,1,Mama's Beste,Inactive,...,4000.0,4000000.0,300.0,20000.0,30000000.0,1,334,Apparel - Active - Women - Jackets & Coats - P...,True,True
3,1,Foursource Buying SE,Germany,Berlin,Inactive,2017-02-01 16:05:47.881287+00:00,2017M02,1,Mama's Beste,Inactive,...,4000.0,4000000.0,300.0,20000.0,30000000.0,1,2045,Apparel - Casual - Kids - Trousers - Pants,True,True
4,1,Foursource Buying SE,Germany,Berlin,Inactive,2017-02-01 16:05:47.881287+00:00,2017M02,1,Mama's Beste,Inactive,...,4000.0,4000000.0,300.0,20000.0,30000000.0,1,361,Apparel - Active - Women - Trousers - Bib Shorts,True,True
5,1,Foursource Buying SE,Germany,Berlin,Inactive,2017-02-01 16:05:47.881287+00:00,2017M02,1,Mama's Beste,Inactive,...,4000.0,4000000.0,300.0,20000.0,30000000.0,1,578,Apparel - Casual - Kids - Pullover & Sweater -...,True,True
6,1,Foursource Buying SE,Germany,Berlin,Inactive,2017-02-01 16:05:47.881287+00:00,2017M02,1,Mama's Beste,Inactive,...,4000.0,4000000.0,300.0,20000.0,30000000.0,1,1182,Apparel - Formal - Kids - Dresses & Robes - Gown,True,True
7,1,Foursource Buying SE,Germany,Berlin,Inactive,2017-02-01 16:05:47.881287+00:00,2017M02,1,Mama's Beste,Inactive,...,4000.0,4000000.0,300.0,20000.0,30000000.0,1,1346,Apparel - Active - Women - Pullover & Sweater ...,True,True
8,1,Foursource Buying SE,Germany,Berlin,Inactive,2017-02-01 16:05:47.881287+00:00,2017M02,13,The Wall,Inactive,...,NaN,NaN,NaN,NaN,NaN,13,1105,Apparel - Workwear - Kids - Accessories - Gloves,True,True
9,2,Best Blue Mode GmbH,Germany,,Active,2017-02-01 18:18:47.532717+00:00,2017M02,2,Meru,Active,...,245.0,270.0,500.0,500.0,1000.0,2,341,Apparel - Active - Women - Jackets & Coats - W...,True,True


In [14]:
m_sql = """
SELECT
m.id_manufacturer
, m.name AS manufacturer_name
, c.name AS manufacturer_country
, m.city AS manufacturer_city
, CASE WHEN pf.filled_mandatory_reached_at IS NOT NULL THEN 'onboarded' ELSE 'not-onboarded' END AS onboarded
, m.created_at AS manufacturer_created_at
, to_char(m.created_at::TIMESTAMP WITH TIME ZONE, 'yyyy"M"mm') AS cohort
, pf.filled_mandatory_reached_at AS onboarded_at
, po.min_qty AS po_min
, po.avg_qty AS po_avg
, po.max_qty AS po_max
, SUM(CASE WHEN ltt.id_lead_time_type = 1 THEN lt.minimum END) AS onstock_min_lead_time
, SUM(CASE WHEN ltt.id_lead_time_type = 1 THEN lt.maximum END) AS onstock_max_lead_time
, SUM(CASE WHEN ltt.id_lead_time_type = 2 THEN lt.minimum END) AS made2order_min_lead_time
, SUM(CASE WHEN ltt.id_lead_time_type = 2 THEN lt.maximum END) AS made2order_max_lead_time
, SUM(CASE WHEN ltt.id_lead_time_type = 3 THEN lt.minimum END) AS sample_min_lead_time
, SUM(CASE WHEN ltt.id_lead_time_type = 3 THEN lt.maximum END) AS sample_max_lead_time
FROM manufacturer AS m
LEFT JOIN country AS c ON c.id_country = m.fk_country
LEFT JOIN profile_fill AS pf ON pf.fk_manufacturer = m.id_manufacturer
LEFT JOIN purchase_order_size AS po ON po.fk_manufacturer = m.id_manufacturer
LEFT JOIN lead_time AS lt ON m.id_manufacturer = lt.fk_manufacturer
LEFT JOIN lead_time_type AS ltt ON lt.fk_lead_time_type = ltt.id_lead_time_type
WHERE m.fk_manufacturer_status = 1
GROUP BY m.id_manufacturer, c.id_country, pf.id_profile_fill, po.id_purchase_order_size
ORDER BY m.id_manufacturer;
"""
m_df = helper.query_retrieval(m_sql, manufacturer_access)

In [15]:
m_df[:5]


,id_manufacturer,manufacturer_name,manufacturer_country,manufacturer_city,onboarded,manufacturer_created_at,cohort,onboarded_at,po_min,po_avg,po_max,onstock_min_lead_time,onstock_max_lead_time,made2order_min_lead_time,made2order_max_lead_time,sample_min_lead_time,sample_max_lead_time
0,2,Esstee Exports,India,Tirupur,onboarded,2017-02-01 17:49:51.044211+00:00,2017M02,2017-02-01 12:00:00+00:00,1000.0,3000.0,50000.0,45.0,60.0,60.0,90.0,15.0,30.0
1,4,Eastman Exports Global Clothing Pvt Ltd,India,Tirupur,onboarded,2017-02-01 18:17:49.035155+00:00,2017M02,2017-02-04 12:00:00+00:00,3000.0,50000.0,400000.0,50.0,70.0,60.0,90.0,20.0,30.0
2,5,Saga Holdings,China,Nanjing,onboarded,2017-02-19 13:04:33.020349+00:00,2017M02,2017-10-30 14:58:47+00:00,500.0,2000.0,500000.0,47.0,59.0,80.0,90.0,5.0,9.0
3,6,BKV,Slovakia,,onboarded,2017-02-23 16:11:58.875105+00:00,2017M02,2017-10-19 14:17:17+00:00,1.0,20.0,55000.0,30.0,60.0,60.0,90.0,14.0,30.0
4,7,S4 Fashion Partner,Vietnam,,onboarded,2017-02-24 10:49:29.908933+00:00,2017M02,2017-05-31 12:00:00+00:00,500.0,1000.0,30000.0,30.0,30.0,60.0,90.0,7.0,14.0
